In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
import matplotlib.pyplot as plt
import pickle as pk
import joblib

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix
# from sklearn.metrics import plot_confusion_matrix



In [2]:
X = pd.read_csv("data_preprocessed_python/eda_data.csv")
Y = pd.read_csv("data_preprocessed_python/eda_labels.csv")
# print(X)
print(np.mean(Y))

Unnamed: 0     19.500000
Arousal         5.156711
AVG_Arousal     4.977347
STD_Arousal     1.832603
dtype: float64


In [3]:
print(X.eda_raw[0])

[-637.8723148573772, -615.5655798498192, -615.5655798498192, -615.5655798498192, -611.1039167713607, -602.1802745076711, -602.1802745076711, -602.1802745076711, -602.1802745076711, -584.3317254485446, -584.3317254485446, -584.3317254485446, -584.3317254485446, -566.4814901479986, -566.4814901479986, -566.4814901479986, -566.4814901479986, -553.0927069329191, -553.0927069329191, -553.0927069329191, -553.0927069329191, -544.1663243813091, -544.1663243813091, -544.1663243813091, -539.7029749717913, -526.3122941637994, -526.3122941637994, -526.3122941637994, -526.3122941637994, -517.3846464381204, -517.3846464381204, -517.3846464381204, -517.3846464381204, -508.4565769280889, -508.4565769280889, -508.4565769280889, -508.4565769280889, -499.52808560361154, -499.52808560361154, -499.52808560361154, -495.06368175172247, -486.13455764972605, -486.13455764972605, -486.13455764972605, -486.13455764972605, -486.13455764972605, -486.13455764972605, -486.13455764972605, -486.13455764972605, -477.20

In [4]:
X.columns

Index(['Unnamed: 0', 'eda_filtered', 'eda_raw', 'kurt', 'max_amp', 'max_diff',
       'max_eda', 'mean_amp', 'mean_eda', 'median_eda', 'min_eda', 'num_peaks',
       'onsets', 'peaks', 'rise_time', 'skewness', 'std_dev', 'variance'],
      dtype='object')

In [5]:
X.rename(columns={"Unnamed: 0":"video_num"}, inplace=True)
Y.rename(columns={"Unnamed: 0":"video_num"}, inplace=True)
X.columns
Y.columns

Index(['video_num', 'Arousal', 'AVG_Arousal', 'STD_Arousal'], dtype='object')

In [6]:
Y_label = pd.DataFrame((Y['Arousal'])) #.astype(int)
# Y_label['AVG_Arousal'] = np.where((Y_label['AVG_Arousal'] < 6.0), np.rint(Y_label['AVG_Arousal']), np.ceil(Y_label['AVG_Arousal']))
# Y_label_avg = pd.DataFrame(Y['AVG_Arousal'])
Y_label
X_feature = X.drop(columns=['eda_raw','video_num','eda_filtered','onsets','peaks','variance','mean_eda'])
X_features = X_feature[X_feature.notnull().all(axis=1)] #dropna(inplace=True)
# X_features = np.array(X_features['std_dev']).reshape(-1,1)
Y_labels = Y_label[X_feature.notnull().all(axis=1)]
# Y_labels['AVG_Arousal'] = Y_labels['AVG_Arousal'].astype(int)
# Y_labels_ex = Y_labels[(Y_labels['Arousal'] <= 3.0) | (Y_labels['Arousal'] >= 6.0)]
Y_labels['classes'] = np.where((Y_labels['Arousal'] <= 3.0), -1, (np.where((Y_labels['Arousal'] <= 6.0), 0, 1)))

# X_features = X_features[Y_labels.classes != 0]
# Y_labels = Y_labels[Y_labels.classes != 0]

print(len(Y_labels[Y_labels.classes == -1]))
print(len(Y_labels[Y_labels.classes == 0]))
print(len(Y_labels[Y_labels.classes == 1]))


# Y_labels_ex.drop(columns=['Arousal'], inplace=True)
# X_features_ex = X_features[(Y_labels['Arousal'] <= 3.0) | (Y_labels['Arousal'] >= 6.0)]
# X_features_ex
# Y_labels_ex

217
513
511


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_features, Y_labels['classes'], test_size=0.30, random_state=10)
# X_train.join(y_train)
print(X_train.head())

print("\n testing data: \n")
print(len(y_test[y_test==-1]))
print(len(y_test[y_test==0]))
print(len(y_test[y_test==1]))

print("\n training data: \n")
print(len(y_train[y_train==-1]))
print(len(y_train[y_train==0]))
print(len(y_train[y_train==1]))
print(X_train.columns)

         kurt       max_amp      max_diff      max_eda     mean_amp  \
245 -1.031018   5295.989552   9542.340425  9050.925379  5415.250795   
944 -0.018622    232.161011    462.925122   329.674332   129.727947   
732 -0.777564  29571.467437  45804.222036  4967.198328 -3909.582845   
276 -1.414127   1628.542301   3210.612545   622.574083  -989.548698   
361 -0.805845  10561.714076  20532.683881  7282.097857 -1054.043972   

       median_eda       min_eda  num_peaks   rise_time  skewness       std_dev  
245   4799.250278   -491.415046          5  115.400000 -0.177598   2534.439686  
944    104.149898   -133.250790         36  106.888889  0.029346     80.272181  
732 -10665.424454 -40837.023708          6  289.500000 -0.477261  12254.660106  
276  -1005.714208  -2588.038463         12  233.833333 -0.134769    966.895409  
361  -2711.509086 -13250.586024          3  562.000000 -0.026996   5239.229506  

 testing data: 

64
144
165

 training data: 

153
369
346
Index(['kurt', 'max_amp', '

In [8]:
#with feature scaling

scalar = StandardScaler()
scalar.fit(X_train,y_train)
scaled_X_train = scalar.transform(X_train)
scaled_X_test = scalar.transform(X_test)
joblib.dump(scalar, 'eda_scaler.pkl', protocol=2)
# fs_log_reg = LogisticRegression(C=10).fit(scaled_X_train,y_train)

# #training data accuracy 
# #removing rise time causes much less units to be classified is low arousal
# print("Training Data: \n")
# fs_y_pred = fs_log_reg.predict(scaled_X_train)
# R = recall_score(y_train,fs_y_pred,average=None)
# P = precision_score(y_train,fs_y_pred,average=None)
# F1 = f1_score(y_train,fs_y_pred,average=None)
# print("Recall: "+str(R),"Precision: "+str(P), "F1 score: "+str(F1))
# print(fs_log_reg.coef_, fs_log_reg.score(scaled_X_train,y_train))
# # print(X_features.columns)

# matrix = confusion_matrix(fs_y_pred, y_train)

# print(matrix)

# #testing data accuracy 
# print("\n Testing Data: \n")
# fs_y_pred = fs_log_reg.predict(scaled_X_test)
# R = recall_score(y_test,fs_y_pred,average=None)
# P = precision_score(y_test,fs_y_pred,average=None)
# F1 = f1_score(y_test,fs_y_pred,average=None)
# print("Recall: "+str(R),"Precision: "+str(P), "F1 score: "+str(F1))
# print(fs_log_reg.coef_, fs_log_reg.score(scaled_X_test,y_test))
# # print(X_features.columns)

# confusion_matrix(fs_y_pred, y_test)



['eda_scaler.pkl']

In [9]:
from sklearn.neural_network import MLPClassifier

nodes = [i for i in range(1,len(X_train.columns)*2)]
layers = [i for i in range(1,len(X_train.columns)*2)]

outputs = pd.DataFrame(columns=['layers','nodes','test_acc','train_acc', 'test_precision','test_recall','test_F1', 'train_precision','train_recall','train_F1'])

lowest_acc_diff = 1.0
lowest_acc_diff_nodes = 1
lowest_acc_diff_layers = 1

best_train_acc = 0.0
best_train_nodes = 1
best_train_layers = 1

best_test_acc = 0.0
best_test_nodes = 1
best_test_layers = 1

for i in range(len(layers)):
    for j in range(len(nodes)):
#         print("layers: "+str(layers[i])+", nodes: "+str(nodes[j])+" \n")
        clf = MLPClassifier(solver='lbfgs',activation='tanh', learning_rate='invscaling', learning_rate_init = 0.0001,hidden_layer_sizes=(nodes[j], layers[i]), random_state=12)
        clf.fit(scaled_X_train, y_train)
        
#         print("training data: \n")
        preds = clf.predict(scaled_X_train)
        trn_acc = clf.score(scaled_X_train, y_train)
        R_train = recall_score(y_train,preds,average=None)
        P_train = precision_score(y_train,preds,average=None)
        print(set(y_train)-set(preds))
        F1_train = f1_score(y_train,preds,average=None)
        if trn_acc > best_train_acc:
            best_train_acc = trn_acc
            best_train_nodes = nodes[j]
            best_train_layers = layers[i]
#         print(confusion_matrix(preds, y_train))

#         print("\n testing data: \n")
        preds = clf.predict(scaled_X_test)
        tst_acc = clf.score(scaled_X_test, y_test)
        R_test = recall_score(y_test,preds,average=None)
        P_test = precision_score(y_test,preds,average=None)
        print(set(y_test)-set(preds))
        F1_test = f1_score(y_test,preds,average=None)
        if tst_acc > best_test_acc:
            best_test_acc = tst_acc
            best_test_nodes = nodes[j]
            best_test_layers = layers[i]
            
#         if abs(trn_acc-tst_acc) < lowest_acc_diff:
#             lowest_acc_diff = abs(trn_acc-tst_acc)
#             lowest_acc_diff_nodes = nodes[j]
#             lowest_acc_diff_layers = layers[i]
        
        outputs = outputs.append(pd.DataFrame([[layers[i],nodes[j],tst_acc,trn_acc,P_test,R_test,F1_test,P_train,R_train,F1_train]], columns=['layers','nodes','test_acc','train_acc', 'test_precision','test_recall','test_F1', 'train_precision','train_recall','train_F1']))
#         print(outputs.head)
#         print(confusion_matrix(preds, y_test))
        
#         print("Train accuracy: "+str(trn_acc)+ ", Test accuracy: "+ str(tst_acc)+ " \n")
    
# best accuracy on training data


# lowest difference in accuracy between training and testing data sets
# print("\n lowest difference between accuries was "+str(lowest_acc_diff) +" using "+str( lowest_acc_diff_layers)+" layers with "+str(lowest_acc_diff_nodes)+ " nodes each")

# clf = MLPClassifier(solver='lbfgs',activation='tanh',hidden_layer_sizes=(lowest_acc_diff_nodes, lowest_acc_diff_layers), random_state=42)
# clf.fit(scaled_X_train, y_train)

# preds = clf.predict(scaled_X_train)
# print(confusion_matrix(y_train, preds))
# R = recall_score(y_train,preds,average=None)
# P = precision_score(y_train,preds,average=None)
# F1 = f1_score(y_train,preds,average=None)
# print("Training Recall: "+str(R),"Training Precision: "+str(P), "Training F1 score: "+str(F1))
# print("Training accuracy is "+str(clf.score(scaled_X_train, y_train)) +" \n")

# preds = clf.predict(scaled_X_test)
# print(confusion_matrix(y_test, preds))
# R = recall_score(y_test,preds,average=None)
# P = precision_score(y_test,preds,average=None)
# F1 = f1_score(y_test,preds,average=None)
# print("Testing Recall: "+str(R),"Testing Precision: "+str(P), "Testing F1 score: "+str(F1))



/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


{1, -1}
{1, -1}
set()
set()
set()
set()
{-1}
{-1}


/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py

{-1}
{-1}
set()
set()
{-1}
{-1}


/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py

...

set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
{-1}
{-1}


/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
{-1}
{-1}


/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set(

In [10]:
# print("best training accuracy is "+str(best_train_acc)+" using "+ str(best_train_layers)+" layers with "+ str(best_train_nodes) + " nodes each")
# np.set_printoptions(precision=2)
class_names = ['low', 'mod', 'hi']

clf = MLPClassifier(solver='lbfgs',activation='tanh',learning_rate='invscaling', hidden_layer_sizes=(16, 21), random_state=12)
clf.fit(scaled_X_train, y_train)
preds = clf.predict(scaled_X_train)

mtx = confusion_matrix(y_train, preds).astype('float')
for i in range(len(mtx)):
#     row = mtx[i]
    mtx[i] = (mtx[i])/np.sum(mtx[i])

print(mtx)
print(confusion_matrix(y_train, preds))

joblib.dump(clf, 'nn_eda_model.pkl', protocol=2)

R = recall_score(y_train,preds,average=None)
P = precision_score(y_train,preds,average=None)
F1 = f1_score(y_train,preds,average=None)
print("Training Recall: "+str(R),"Training Precision: "+str(P), "Training F1 score: "+str(F1))
print(clf.score(scaled_X_train,y_train))


# best testing accuracy

preds = clf.predict(scaled_X_test)
# print(confusion_matrix(y_test, preds))
mtx = confusion_matrix(y_test, preds).astype('float')
for i in range(len(mtx)):
#     row = mtx[i]
    mtx[i] = (mtx[i])/np.sum(mtx[i])

print(mtx)
print(confusion_matrix(y_test, preds))


R = recall_score(y_test,preds,average=None)
P = precision_score(y_test,preds,average=None)
F1 = f1_score(y_test,preds,average=None)
print("Testing Recall: "+str(R),"Testing Precision: "+str(P), "Testing F1 score: "+str(F1))
print("Testing accuracy is "+str(clf.score(scaled_X_test, y_test))+" \n")


# # best accuracy on testing data
# print("best testing accuracy is "+str(best_test_acc)+" using "+ str(best_test_layers)+" layers with "+ str(best_test_nodes) + " nodes each")
# clf = MLPClassifier(solver='lbfgs',activation='tanh', learning_rate='invscaling', hidden_layer_sizes=(best_test_nodes, best_test_layers), random_state=42)
# clf.fit(scaled_X_train, y_train)

# preds = clf.predict(scaled_X_train)
# print(confusion_matrix(y_train, preds))
# R = recall_score(y_train,preds,average=None)
# P = precision_score(y_train,preds,average=None)
# F1 = f1_score(y_train,preds,average=None)
# print("Training Recall: "+str(R),"Training Precision: "+str(P), "Training F1 score: "+str(F1))
# print("Training accuracy is "+str(clf.score(scaled_X_train, y_train)) +" \n")

# preds = clf.predict(scaled_X_test)
# print(confusion_matrix(y_test, preds))
# R = recall_score(y_test,preds,average=None)
# P = precision_score(y_test,preds,average=None)
# F1 = f1_score(y_test,preds,average=None)
# print("Testing Recall: "+str(R),"Testing Precision: "+str(P), "Testing F1 score: "+str(F1))

[[0.66666667 0.16993464 0.16339869]
 [0.04336043 0.83468835 0.12195122]
 [0.05202312 0.15895954 0.78901734]]
[[102  26  25]
 [ 16 308  45]
 [ 18  55 273]]
Training Recall: [0.66666667 0.83468835 0.78901734] Training Precision: [0.75       0.79177378 0.79591837] Training F1 score: [0.70588235 0.81266491 0.79245283]
0.7868663594470046
[[0.34375    0.375      0.28125   ]
 [0.1875     0.4375     0.375     ]
 [0.2        0.35757576 0.44242424]]
[[22 24 18]
 [27 63 54]
 [33 59 73]]
Testing Recall: [0.34375    0.4375     0.44242424] Testing Precision: [0.26829268 0.43150685 0.50344828] Testing F1 score: [0.30136986 0.43448276 0.47096774]
Testing accuracy is 0.42359249329758714 



In [11]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
outputs.sort_values(by=['train_acc','test_acc'], ascending=False)

,layers,nodes,test_acc,train_acc,test_precision,test_recall,test_F1,train_precision,train_recall,train_F1
0,17,17,0.372654,0.868664,"[0.22077922077922077, 0.4064516129032258, 0.41...","[0.265625, 0.4375, 0.3575757575757576]","[0.24113475177304966, 0.4214046822742475, 0.38...","[0.7866666666666666, 0.8877005347593583, 0.883...","[0.7712418300653595, 0.8997289972899729, 0.878...","[0.778877887788779, 0.8936742934051144, 0.8811..."
0,21,20,0.404826,0.853687,"[0.20689655172413793, 0.4129032258064516, 0.46...","[0.1875, 0.4444444444444444, 0.45454545454545453]","[0.19672131147540986, 0.4280936454849499, 0.46...","[0.7916666666666666, 0.9095890410958904, 0.819...","[0.6209150326797386, 0.8997289972899729, 0.907...","[0.695970695970696, 0.9046321525885559, 0.8614..."
0,14,21,0.428954,0.852535,"[0.24390243902439024, 0.4383561643835616, 0.46...","[0.15625, 0.4444444444444444, 0.5212121212121212]","[0.19047619047619047, 0.4413793103448276, 0.49...","[0.883495145631068, 0.8879781420765027, 0.8120...","[0.5947712418300654, 0.8807588075880759, 0.936...","[0.7109374999999999, 0.8843537414965986, 0.869..."
0,21,19,0.436997,0.844470,"[0.25, 0.46405228758169936, 0.48717948717948717]","[0.25, 0.4930555555555556, 0.46060606060606063]","[0.25, 0.4781144781144781, 0.4735202492211838]","[0.8076923076923077, 0.8643617021276596, 0.837...","[0.6862745098039216, 0.8807588075880759, 0.875...","[0.7420494699646644, 0.8724832214765101, 0.855..."
0,19,20,0.410188,0.844470,"[0.21621621621621623, 0.4370860927152318, 0.47...","[0.25, 0.4583333333333333, 0.4303030303030303]","[0.2318840579710145, 0.44745762711864406, 0.45...","[0.7984496124031008, 0.8693333333333333, 0.835...","[0.673202614379085, 0.8834688346883469, 0.8786...","[0.7304964539007093, 0.8763440860215055, 0.856..."
0,20,15,0.423592,0.837558,"[0.25, 0.43352601156069365, 0.47297297297297297]","[0.203125, 0.5208333333333334, 0.4242424242424...","[0.22413793103448276, 0.47318611987381703, 0.4...","[0.7933884297520661, 0.8505154639175257, 0.838...","[0.6274509803921569, 0.8943089430894309, 0.869...","[0.7007299270072992, 0.8718626155878468, 0.853..."
0,18,21,0.407507,0.836406,"[0.19117647058823528, 0.4506172839506173, 0.46...","[0.203125, 0.5069444444444444, 0.4]","[0.196969696969697, 0.477124183006536, 0.42857...","[0.8473282442748091, 0.8276762402088773, 0.841...","[0.7254901960784313, 0.8590785907859079, 0.861...","[0.7816901408450704, 0.8430851063829788, 0.851..."
0,19,19,0.426273,0.835253,"[0.2708333333333333, 0.4367816091954023, 0.463...","[0.203125, 0.5277777777777778, 0.4242424242424...","[0.23214285714285712, 0.4779874213836478, 0.44...","[0.75, 0.8257575757575758, 0.8764367816091954]","[0.6078431372549019, 0.8861788617886179, 0.881...","[0.671480144404332, 0.8549019607843139, 0.8789..."
0,18,19,0.402145,0.834101,"[0.24489795918367346, 0.39622641509433965, 0.4...","[0.1875, 0.4375, 0.45454545454545453]","[0.21238938053097342, 0.4158415841584158, 0.45...","[0.7364341085271318, 0.8489583333333334, 0.853...","[0.6209150326797386, 0.8834688346883469, 0.875...","[0.6737588652482269, 0.8658698539176627, 0.864..."
0,20,19,0.418231,0.832949,"[0.3076923076923077, 0.42028985507246375, 0.45...","[0.3125, 0.4027777777777778, 0.4727272727272727]","[0.31007751937984496, 0.41134751773049644, 0.4...","[0.7661290322580645, 0.8413098236775819, 0.847...","[0.6209150326797386, 0.9051490514905149, 0.849...","[0.6859205776173285, 0.8720626631853787, 0.848..."


In [12]:

degrees = [1, 2, 3, 4, 5, 6, 7, 8, 9]

def PolyLogRegression(degree, X_train, y_train, X_test, y_test):
    poly = PolynomialFeatures(degree)
    
    #first use training data to fit features
    poly.fit(X_train, y_train)
    #transform training data to create new features
    tr_X_train = poly.transform(X_train)
    #transform testing data to create new features
    tr_X_test = poly.transform(X_test)

    #logistic regression
    clf = LogisticRegression(C=10).fit(tr_X_train, y_train)
#     print(poly.get_feature_names(X_train.columns))
    print(confusion_matrix(y_train, clf.predict(tr_X_train)))
    return clf.score(tr_X_train,y_train)

# for d in degrees:
#     print(d, PolyLogRegression(d, scaled_X_train, y_train, scaled_X_test, y_test))

In [13]:
# from sklearn import linear_model # Using sklearn Perceptron classifier
# from sklearn import ensemble # Using RandomForest classifier
# from sklearn import neighbors # Using nearest neighbors classifier
# from sklearn.model_selection import cross_val_score
# learners = {'Perceptron': linear_model.Perceptron(max_iter=10), 'RandomForest': ensemble.RandomForestClassifier(), 'kNN': neighbors.KNeighborsClassifier(), 'LogisticRegression': LogisticRegression() }

# for classifierName in learners:
#     score = cross_val_score(learners[classifierName],scaled_X_train, y_train, cv=5)
#     print('Accuracy of ' + classifierName + ':\t' + str((score)))